In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!nvidia-smi

Mon Jan  9 12:38:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from bs4 import BeautifulSoup, NavigableString
import requests

import pandas as pd
import re

URL = 'https://stihipushkina.ru/vse-stixi-pushkina-na-odnoj-stranice/'
page = requests.get(URL)
page_decode = page.content.decode('utf-8')
page_decode = re.sub('<br />', ' ', page_decode)
# page_decode.replace('<br />', ' ')
soup = BeautifulSoup(page_decode, 'html.parser')

In [ ]:
page_decode[0:20000]

In [ ]:
# invalid_tags = ['b', 'i', 'u']
# for tag in invalid_tags:
#     for match in soup.find_all(tag):
#         match.unwrap()
        
for match in soup.find_all('span'):
    ''.join(match.strings)

        
poem_titles = soup.find_all('span')
poem_titles = [poem.text for poem in poem_titles]

In [ ]:
poem_titles = poem_titles[1:835]

In [ ]:
poem_titles[0:10]

['Воспоминания в Царском Селе',
 'Старик',
 'Любопытный',
 'Лицинию',
 'Роза',
 'Гроб Анакреона',
 'Разлука (Когда пробил последний счастью час)',
 'Певец',
 'К Морфею',
 'Друзьям (Богами вам еще даны)']

In [ ]:
def extract_between(current, end):
    while current and current != end:
        if isinstance(current, NavigableString):
            text = current.strip()
            if len(text):
                yield text
        current = current.next_element

In [ ]:
poem_text = []
for i in range(1, len(poem_titles) - 1):
    name = poem_titles[i]
    text = ' '.join(text for text in extract_between(soup.find_all('span')[i], 
                                                       soup.find_all('span')[i+1]))
    # text = text.replace(name, '')
    ll = re.sub(name, ' ', text)
    print(name)
    print(ll)
    poem_text.append(ll)

In [ ]:
re.sub(poem_titles[3], '', poem_text[3])

' Лициний, зришь ли ты: на быстрой колеснице, \nВенчанный лаврами, в блестящей багрянице, \nСпесиво развалясь, Ветулий молодой \nВ толпу народную летит по мостовой? \nСмотри, как все пред ним смиренно спину клонят; \nСмотри, как ликторы народ несчастный гонят! \nЛьстецов, сенаторов, прелестниц длинный ряд \nУмильно вслед за ним стремит усердный взгляд; \nЖдут, ловят с трепетом улыбки, глаз движенья, \nКак будто дивного богов благословенья; \nИ дети малые и старцы в сединах, \nВсе ниц пред идолом безмолвно пали в прах: \nДля них и след колес, в грязи напечатленный, \nЕсть некий памятник почетный и священный. О Ромулов народ, скажи, давно ль ты пал? \nКто вас поработил и властью оковал? \nКвириты гордые под иго преклонились. \nКому ж, о небеса, кому поработились? \n(Скажу ль?) Ветулию! Отчизне стыд моей, \nРазвратный юноша воссел в совет мужей; \nЛюбимец деспота сенатом слабым правит, \nНа Рим простер ярем, отечество бесславит; \nВетулий римлян царь!.. О стыд, о времена! \nИли вселенная 

In [ ]:
for i in range(len(poem_text)-1):
  txt = poem_text[i]
  txt = re.sub(poem_titles[i], '', txt)
  poem_text[i] = txt

In [ ]:
df = pd.DataFrame(list(zip(poem_titles[:-1], poem_text[:-1])), columns=['title', 'text'])

In [ ]:
df.head(10)

,title,text
0,Воспоминания в Царском Селе,Навис покров угрюмой нощи \nНа своде дремлющи...
1,Старик,"Уж я не тот любовник страстный, \nКому дивилс..."
2,Любопытный,"— Что ж нового? «Ей-богу, ничего». \n— Эй, не..."
3,Лицинию,"Лициний, зришь ли ты: на быстрой колеснице, \..."
4,Роза,"Где наша роза, \nДрузья мои? \nУвяла роза, \n..."
5,Гроб Анакреона,Все в таинственном молчанье; \nХолм оделся те...
6,Разлука (Когда пробил последний счастью час),Разлука (Когда пробил последний счастью час) К...
7,Певец,Слыхали ль вы за рощей глас ночной \nПевца лю...
8,К Морфею,"Морфей, до утра дай отраду \nМоей мучительной..."
9,Друзьям (Богами вам еще даны),Друзьям (Богами вам еще даны) Богами вам ещё д...


In [ ]:
def clean_poem_text(poem_text):
    poem_text = poem_text.strip()
    poem_text = re.sub(' +', ' ', poem_text)
    poem_text = re.sub('\n\n\r\n\r\n', '\n\r\n \n\r\n', poem_text)
    poem_text = re.sub('\n\r\n', '\n', poem_text)
    poem_text = re.sub(' +', ' ', poem_text)
    return poem_text


In [ ]:
def split_into_stanzas(poem_title_list, poem_text_list):
    return_title_list = []
    return_stanza_list = []
    for poem_index in range(len(poem_title_list)):
        poem_text = clean_poem_text(poem_text_list[poem_index])
        if poem_title_list[poem_index] == 'The Bells':
            first_split = poem_text.split('IV')
            stanza_four = first_split[1]
            second_split = first_split[0].split('III')
            stanza_three = second_split[1]
            third_split = second_split[0].split('II')
            stanza_two = third_split[1]
            stanza_one = third_split[0]
            poem_stanzas = [stanza_one, stanza_two, stanza_three, stanza_four]
        else:
            poem_stanzas = poem_text.split('\n \n')
        for stanza in poem_stanzas:
            return_title_list.append(poem_title_list[poem_index])
            return_stanza_list.append(stanza)
    return return_title_list, return_stanza_list

In [ ]:
stanza_title, stanza_text = split_into_stanzas(df['title'].values, df['text'].values)

In [ ]:
stanza_df = pd.DataFrame(list(zip(stanza_title, stanza_text)), columns=['title', 'stanza_text'])

In [ ]:
def split_into_lines(poem_title_list, poem_text_list):
    return_title_list = []
    return_line_list = []
    for poem_index in range(len(poem_title_list)):
        poem_lines = poem_text_list[poem_index].split('\n')
        for line in poem_lines:
            return_title_list.append(poem_title_list[poem_index])
            return_line_list.append(line.strip())
    return return_title_list, return_line_list
        

In [ ]:
line_title, line_text = split_into_lines(stanza_df['title'].values, stanza_df['stanza_text'].values)

In [ ]:
line_df = pd.DataFrame(list(zip(line_title, line_text)), columns=['title', 'line_text'])

In [ ]:
line_df.to_csv('pushkin_poems_lines.csv', index=False)
stanza_df.to_csv('puchkin_poems_stanzas.csv', index=False)

In [ ]:
%mv /content/pushkin_poems_lines.csv  /content/drive/MyDrive/pushkin_ai/pushkin_poems_lines.csv